In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

In [2]:
from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM


C:\Users\matan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [3]:
#need to shuffle to respect iid principal for when we split the data
de = DataExploration()

twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()

(22223, 10)


(19276, 10)
(41499, 10)


In [4]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    twitter_df, 'twitter')

(41499, 10)


In [5]:
best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7}
dt_clf = DecisionTree(model_name='dt_wip1', dataset_name='spam_twitter', X_test=X_test, y_test=y_test, best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth


[[3504  378]
 [ 316 4102]]
             precision    recall  f1-score   support

         -1       0.92      0.90      0.91      3882
          1       0.92      0.93      0.92      4418

avg / total       0.92      0.92      0.92      8300



AttributeError: 'Pipeline' object has no attribute 'best_estimator_'

In [6]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)

{'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7}


In [7]:
# dnn = DNN(n_classes=2, model_name='dnn_wip_3', dataset_name='spam_twitter')
# dnn.train(
#     learning_rate=0.05,
#     steps=1000,
#     batch_size=30,
#     hidden_units=[100, 100],
#     training_examples=X_train_nn,
#     training_targets=y_train_nn,
#     validation_examples=X_validation_nn,
#     validation_targets=y_validation_nn)
# 
# y_pred = dnn.predict(X_test, y_test)

In [8]:
best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
# todo this is lower than pre pruning, figure out why
boosting_clf = Boosting(model_name='boosting_wip0', dataset_name='spam_twitter', X_test=X_test, y_test=y_test,
                        best_params=best_params)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.910):
{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}


[[3419  463]
 [ 522 3896]]
             precision    recall  f1-score   support

         -1       0.87      0.88      0.87      3882
          1       0.89      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [13]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)

{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}


In [ ]:
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}
knn_clf = KNN(model_name='knn_wip0', dataset_name='spam_twitter', best_params=best_params)
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.880):
{'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}


[[3453  429]
 [ 531 3887]]
             precision    recall  f1-score   support

         -1       0.87      0.89      0.88      3882
          1       0.90      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [9]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)

{'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}


In [10]:
best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
               'loss': 'mean_squared_error'}

keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='spam_twitter', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Fitting 3 folds for each of 600 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.4min


[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 24.9min


[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 213.4min


[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 285.2min


...

C:\Users\matan\Anaconda3\lib\site-packages\sklearn\base.py:115: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\base.py:115: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\base.py:115: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


[[   0 3882]
 [   0 4418]]
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00      3882
          1       0.53      1.00      0.69      4418

avg / total       0.28      0.53      0.37      8300



C:\Users\matan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

{'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005, 'loss': 'mean_squared_error'}


In [8]:
# keras_nn_clf = NN_Keras(model_name='keras_nn_optimizer_wip0', dataset_name='spam_twitter')
# keras_nn_clf.train(X_train, y_train, optimizer_tuning=True)
# y_pred = keras_nn_clf.predict(X_test)
# keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best: 0.896443 using {'optimizer': 'Adam'}
0.509805 (0.035324) with: {'optimizer': 'SGD'}
0.778487 (0.168809) with: {'optimizer': 'RMSprop'}
0.896292 (0.005016) with: {'optimizer': 'Adagrad'}
0.781861 (0.171186) with: {'optimizer': 'Adadelta'}
0.896443 (0.001033) with: {'optimizer': 'Adam'}
0.780054 (0.169650) with: {'optimizer': 'Adamax'}
0.779481 (0.169423) with: {'optimizer': 'Nadam'}


[[   0 3882]
 [   0 4418]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      3882
          1       0.53      1.00      0.69      4418

avg / total       0.28      0.53      0.37      8300



C:\Users\matan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# from sklearn.svm import SVC
# 
# linear_svm_clf = SVC(kernel='linear') #optimize kernel, regularization, gamma
# linear_svm_clf.fit(X_train, y_train)
# y_pred = linear_svm_clf.predict(X_test)

In [16]:

svm_clf = SVM(model_name='svm_wip0', dataset_name='spam_twitter')
svm_clf.train(X_train, y_train)
y_pred = svm_clf.predict(X_test)
svm_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
svm_best_params = svm_clf.get_best_params()
print(svm_best_params)

In [ ]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    speed_dating_df, 'speed_dating')

